##### Copyright 2020 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# feels.like with TensorFlow Lite Model Maker with TensorFlow 2.0

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/examples/blob/master/lite/codelabs/flower_classification/ml/Flower_Classification_with_TFLite_Model_Maker.ipynb">      
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/examples/blob/master/lite/codelabs/flower_classification/ml/Flower_Classification_with_TFLite_Model_Maker.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
</table>

Model Maker library simplifies the process of adapting and converting a TensorFlow neural-network model to particular input data when deploying this model for on-device ML applications.

This notebook shows an end-to-end example that utilizes this Model Maker library to illustrate the adaption and conversion of a commonly-used image classification model to classify flowers on a mobile device.

# Additional Data Sources

DeepFashion (http://mmlab.ie.cuhk.edu.hk/projects/DeepFashion.html)

@inproceedings{liuLQWTcvpr16DeepFashion,
 author = {Liu, Ziwei and Luo, Ping and Qiu, Shi and Wang, Xiaogang and Tang, Xiaoou},
 title = {DeepFashion: Powering Robust Clothes Recognition and Retrieval with Rich Annotations},
 booktitle = {Proceedings of IEEE Conference on Computer Vision and Pattern Recognition (CVPR)},
 month = {June},
 year = {2016} 
 }

## Prerequisites

To run this example, we first need to install serveral required packages, including Model Maker package that in github [repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).

In [2]:
!pip install tflite-model-maker-nightly

     |████████████████████████████████| 593kB 27.5MB/s 
     |████████████████████████████████| 645kB 45.9MB/s 
     |████████████████████████████████| 122kB 35.1MB/s 
     |████████████████████████████████| 6.3MB 48.1MB/s 
     |████████████████████████████████| 1.2MB 38.9MB/s 
     |████████████████████████████████| 1.1MB 37.5MB/s 
     |████████████████████████████████| 112kB 55.4MB/s 
     |████████████████████████████████| 174kB 42.6MB/s 
     |████████████████████████████████| 92kB 14.4MB/s 
     |████████████████████████████████| 1.1MB 41.1MB/s 
     |████████████████████████████████| 849kB 37.8MB/s 
     |████████████████████████████████| 194kB 59.7MB/s 
     |████████████████████████████████| 102kB 13.5MB/s 
     |████████████████████████████████| 358kB 56.1MB/s 
     |████████████████████████████████| 686kB 56.8MB/s 
     |████████████████████████████████| 38.2MB 65kB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115928 sha256=b03052588e949b78c030

Import the required packages.

In [3]:
import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.image_classifier import DataLoader

## Simple End-to-End Example

### Get the data path

In [4]:
import os
os.getcwd()

'/content'

In [5]:
import shutil
shutil.unpack_archive("fashionimgs.zip")
image_path = os.path.join(os.path.dirname('fashionimgs.zip'), 'fashionimgs')

### Run the example
The example just consists of 4 lines of code as shown below, each of which representing one step of the overall process.


1.   Load input data specific to an on-device ML app. Split it to training data and testing data.

In [6]:
data = DataLoader.from_folder(image_path)
#90/5/5 split
train_data, rest_data = data.split(0.9)
validationdata, test_data = rest_data.split(0.5)

INFO:tensorflow:Load image with size: 2560, num_label: 13, labels: 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90.


2. Customize the TensorFlow model.

In [7]:
#define hyperparameters
epoch = 15
dropout = 0.3

#models for transfer learning compatible with TFLite image classification
models = ['efficientnet_lite0','efficientnet_lite1','efficientnet_lite2','efficientnet_lite3','efficientnet_lite4','mobilenet_v2','resnet_50']

In [8]:
for i in models:
  j = str(i)
  print(j)
  model = image_classifier.create(train_data, model_spec=model_spec.get(i), epochs=epoch, dropout_rate=dropout, validation_data=validationdata, shuffle=True)
  loss, accuracy = model.evaluate(test_data)

efficientnet_lite0
INFO:tensorflow:Retraining the models...


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2 (HubKer (None, 1280)              3413024   
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 13)                16653     
Total params: 3,429,677
Trainable params: 16,653
Non-trainable params: 3,413,024
_________________________________________________________________
None
Epoch 1/15


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


72/72 [==============================] - 40s 114ms/step - loss: 2.2589 - accuracy: 0.2708 - val_loss: 1.8668 - val_accuracy: 0.4219
Epoch 2/15
72/72 [==============================] - 8s 106ms/step - loss: 1.9255 - accuracy: 0.3932 - val_loss: 1.8128 - val_accuracy: 0.4375
Epoch 3/15
72/72 [==============================] - 8s 109ms/step - loss: 1.8520 - accuracy: 0.4245 - val_loss: 1.8233 - val_accuracy: 0.4531
Epoch 4/15
72/72 [==============================] - 8s 112ms/step - loss: 1.7892 - accuracy: 0.4462 - val_loss: 1.7781 - val_accuracy: 0.4688
Epoch 5/15
72/72 [==============================] - 9s 119ms/step - loss: 1.7452 - accuracy: 0.4670 - val_loss: 1.7677 - val_accuracy: 0.5000
Epoch 6/15
72/72 [==============================] - 8s 110ms/step - loss: 1.7234 - accuracy: 0.4831 - val_loss: 1.7757 - val_accuracy: 0.4609
Epoch 7/15
72/72 [==============================] - 7s 94ms/step - loss: 1.6852 - accuracy: 0.5052 - val_loss: 1.7726 - val_accuracy: 0.4844
Epoch 8/15
72/72 

INFO:tensorflow:Retraining the models...


...

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2_4 (HubK (None, 1280)              11837936  
_________________________________________________________________
dropout_4 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 13)                16653     
Total params: 11,854,589
Trainable params: 16,653
Non-trainable params: 11,837,936
_________________________________________________________________
None
Epoch 1/15
72/72 [==============================] - 20s 211ms/step - loss: 2.2591 - accuracy: 0.2626 - val_loss: 1.9511 - val_accuracy: 0.4219
Epoch 2/15
72/72 [==============================] - 15s 203ms/step - loss: 1.9212 - accuracy: 0.3945 - val_loss: 1.8740 - val_accuracy: 0.4453
Epoch 3/15
72/72 [==============================] - 15s 202ms/step - loss: 1

INFO:tensorflow:Retraining the models...


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2_5 (HubK (None, 1280)              2257984   
_________________________________________________________________
dropout_5 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 13)                16653     
Total params: 2,274,637
Trainable params: 16,653
Non-trainable params: 2,257,984
_________________________________________________________________
None
Epoch 1/15
72/72 [==============================] - 11s 112ms/step - loss: 2.3767 - accuracy: 0.2617 - val_loss: 2.0752 - val_accuracy: 0.3359
Epoch 2/15
72/72 [==============================] - 8s 109ms/step - loss: 2.0215 - accuracy: 0.3793 - val_loss: 1.9896 - val_accuracy: 0.3672
Epoch 3/15
72/72 [==============================] - 8s 109ms/step - loss: 1.891

INFO:tensorflow:Retraining the models...


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2_6 (HubK (None, 2048)              23564800  
_________________________________________________________________
dropout_6 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 13)                26637     
Total params: 23,591,437
Trainable params: 26,637
Non-trainable params: 23,564,800
_________________________________________________________________
None
Epoch 1/15
72/72 [==============================] - 13s 127ms/step - loss: 2.5767 - accuracy: 0.2522 - val_loss: 2.3381 - val_accuracy: 0.3047
Epoch 2/15
72/72 [==============================] - 9s 120ms/step - loss: 2.3066 - accuracy: 0.3455 - val_loss: 2.2384 - val_accuracy: 0.3203
Epoch 3/15
72/72 [==============================] - 9s 122ms/step - loss: 2.1

3.  Test above with data augmentation by TFLite

In [9]:
for i in models:
  j = str(i)
  print(j)
  model = image_classifier.create(train_data, model_spec=model_spec.get(i), epochs=epoch, dropout_rate=dropout, validation_data=validationdata, use_augmentation=True,  shuffle=True)
  loss, accuracy = model.evaluate(test_data)

efficientnet_lite0
INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2_7 (HubK (None, 1280)              3413024   
_________________________________________________________________
dropout_7 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 13)                16653     
Total params: 3,429,677
Trainable params: 16,653
Non-trainable params: 3,413,024
_________________________________________________________________
None
INFO:tensorflow:Use default resize_bicubic.


INFO:tensorflow:Use default resize_bicubic.


INFO:tensorflow:Use customized resize method bilinear


...

INFO:tensorflow:Use customized resize method bilinear


INFO:tensorflow:Use customized resize method bilinear


Epoch 1/15
72/72 [==============================] - 12s 118ms/step - loss: 2.6324 - accuracy: 0.2400 - val_loss: 2.1211 - val_accuracy: 0.3125
Epoch 2/15
72/72 [==============================] - 8s 111ms/step - loss: 2.4809 - accuracy: 0.2726 - val_loss: 1.9986 - val_accuracy: 0.4531
Epoch 3/15
72/72 [==============================] - 8s 111ms/step - loss: 2.4629 - accuracy: 0.2986 - val_loss: 2.2908 - val_accuracy: 0.3516
Epoch 4/15
72/72 [==============================] - 8s 110ms/step - loss: 2.4249 - accuracy: 0.3008 - val_loss: 2.1570 - val_accuracy: 0.4062
Epoch 5/15
72/72 [==============================] - 8s 111ms/step - loss: 2.3334 - accuracy: 0.3225 - val_loss: 2.0887 - val_accuracy: 0.4219
Epoch 6/15
72/72 [==============================] - 8s 114ms/step - loss: 2.2463 - accuracy: 0.3507 - val_loss: 2.1732 - val_accuracy: 0.3906
Epoch 7/15
72/72 [==============================] - 8s 111ms/step - loss: 2.2928 - accuracy: 0.3329 - val_loss: 2.0585 - val_accuracy: 0.3828
Epoch

INFO:tensorflow:Use customized resize method bilinear


4/4 [==============================] - 1s 96ms/step - loss: 2.3396 - accuracy: 0.3125
